In [7]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/dumping/singlish/singlish.txt
# !wget https://f000.backblazeb2.com/file/malay-dataset/dumping/singlish/sg-news.txt
# !wget https://f000.backblazeb2.com/file/malay-dataset/dumping/twitter/manglish.json
# !wget https://f000.backblazeb2.com/file/malay-dataset/dumping/imda/singlish-text.json

files = ['singlish.txt',
        'sg-news.txt',]

In [8]:
import os
import string

vocabs = list(string.ascii_lowercase + string.digits) + [' ']
directory = './'

In [9]:
import unicodedata
import re
import itertools

def preprocessing_text(string):
    
    string = re.sub(
        'http\\S+|www.\\S+',
        '',
        ' '.join(
            [
                word
                for word in string.split()
                if word.find('#') < 0 and word.find('@') < 0
            ]
        ),
    )
        
    string = unicodedata.normalize('NFC', string.lower())
    string = ''.join([c if c in vocabs else ' ' for c in string])
    string = re.sub(r'[ ]+', ' ', string).strip()
    string = (
        ''.join(''.join(s)[:2] for _, s in itertools.groupby(string))
    )
    return string

In [10]:
texts = []

for f in files:
    print(f)
    with open(os.path.join(directory, f)) as fopen:
        text = list(filter(None, fopen.read().split('\n')))
        texts.extend(text)

singlish.txt
sg-news.txt


In [11]:
import json

with open('bahasa-asr-train-combined.json') as fopen:
    data = json.load(fopen)
    
texts.extend(data['Y'])

with open('bahasa-asr-test.json') as fopen:
    data = json.load(fopen)
    
texts.extend(data['Y'])

In [12]:
with open('manglish.json') as fopen:
    t = json.load(fopen)

In [13]:
len(t)

908120

In [14]:
texts.extend(t)

In [15]:
with open('singlish-text.json') as fopen:
    t = json.load(fopen)
    
texts.extend(t)
len(texts)

27585979

In [16]:
import mp
from tqdm import tqdm

def loop(texts):
    texts, _ = texts
    cleaned_texts = []
    for i in tqdm(range(len(texts))):
        t = preprocessing_text(texts[i])
        if len(t):
            cleaned_texts.append(t)
    return cleaned_texts

cleaned_texts = mp.multiprocessing(texts, loop, cores = 16)

100%|██████████| 1724123/1724123 [04:34<00:00, 6277.80it/s] 


In [17]:
len(texts), len(cleaned_texts)

(27585979, 27355756)

In [18]:
with open('text.txt', 'w') as fopen:
    fopen.write('\n'.join(cleaned_texts))

In [19]:
!./kenlm/build/bin/lmplz --text text.txt --arpa out.arpa -o 3 --prune 0 1 1

=== 1/5 Counting and sorting n-grams ===
Reading /home/husein/malaya-speech/text.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 315567492 types 717175
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:8606100 2:75179687936 3:140961906688
Statistics:
1 717175 D1=0.686594 D2=1.03398 D3+=1.32634
2 9941650/22150543 D1=0.72001 D2=1.06433 D3+=1.36637
3 29758066/94062726 D1=0.674279 D2=1.42124 D3+=1.4373
Memory estimate for binary LM:
type     MB
probing 756 assuming -p 1.5
probing 815 assuming -r models -p 1.5
trie    325 without quantization
trie    188 assuming -q 8 -b 8 quantization 
trie    305 assuming -a 22 array pointer compression
trie    168 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting initial probabilities ===
Chain sizes: 1:8606100 2:15906

In [20]:
!./kenlm/build/bin/build_binary -q 8 -b 7 -a 256 trie out.arpa out.trie.klm

Reading out.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Identifying n-grams omitted by SRI
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Quantizing
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Writing trie
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
SUCCESS


In [21]:
!rm text.txt out.arpa

In [22]:
b2_application_key_id = os.environ['b2_application_key_id']
b2_application_key = os.environ['b2_application_key']

In [23]:
from b2sdk.v1 import *
info = InMemoryAccountInfo()
b2_api = B2Api(info)
application_key_id = b2_application_key_id
application_key = b2_application_key
b2_api.authorize_account("production", application_key_id, application_key)
file_info = {'how': 'good-file'}
b2_bucket = b2_api.get_bucket_by_name('malaya-speech-model')

In [24]:
outPutname = 'language-model/manglish/model.trie.klm'
b2_bucket.upload_local_file(
    local_file='out.trie.klm',
    file_name=outPutname,
    file_infos=file_info,
)